In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import requests, json
from pprint import pprint
import datetime
from datetime import timedelta
from time import sleep
from dateutil import parser
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
pd.set_option('display.max_columns', None)


In [2]:
today = datetime.datetime.today().strftime('%Y-%m-%d')
today

'2022-12-04'

In [3]:
headers={'User-Agent': 'Mozilla/5.0'}

In [4]:
def get_fpl_df():

    team_mapping = {
    1: "Arsenal",
    2: "Aston Villa",
    3: "AFC Bournemouth",
    4: "Brentford",
    5: "Brighton & Hove Albion",
    6: "Chelsea",
    7: "Crystal Palace",
    8: "Everton",
    9: "Fulham",
    11: "Leeds United",
    10: "Leicester City",
    12: "Liverpool",
    13: "Manchester City",
    14: "Manchester United",
    15: "Newcastle United",
    16: "Nottingham Forest",
    17: "Southampton",
    18: "Tottenham Hotspur",
    19: "West Ham United",
    20: "Wolverhampton"
}

    base_url = 'https://fantasy.premierleague.com/api/'
    # get data from bootstrap-static endpoint
    r = requests.request(method = "GET",url=base_url+'bootstrap-static/', proxies={'http': '182.72.150.242:8080'})
    print(r)
    r = r.json()
    # get player data from 'elements' field
    players = r['elements']
    # create players dataframe
    players = pd.json_normalize(r['elements'])
    print(players.iloc[0])
    # get relevant columns
    players = players[['id', 'web_name', 'first_name', 'second_name', 'team', 'element_type']]
    players['full_name'] = players[['first_name', 'second_name']].apply(lambda x: ' '.join(x), axis=1)
    # map team number to team name
    players['team_name'] = players['team'].apply(lambda x: team_mapping[x])

    return players

In [60]:
import aiohttp
from nest_asyncio import asyncio
from fpl import FPL



In [34]:
players = pd.read_csv('../archive/players_fifa23.csv')
players = players[['Name', 'FullName', 'Club']]

teams = pd.read_csv('../archive/teams_fifa23.csv')
teams = teams[['Name', 'League', 'LeagueId']]

In [35]:
pl_teams = teams[teams['LeagueId'] == 13]

In [36]:
pl_players = pd.merge(players, pl_teams, how='inner', left_on='Club', right_on='Name')
pl_players = pl_players.drop(['Name_y', 'LeagueId'], axis=1)

In [37]:
pl_players.rename(columns={'Name_x':'Name'}, inplace=True)

In [38]:
pl_players.sort_values(by=['Club'], inplace=True)

In [39]:
pl_players.to_csv('')

,Name,FullName,Club,League
433,M. Tavernier,Marcus Tavernier,AFC Bournemouth,English Premier League (1)
435,J. Rothwell,Joe Rothwell,AFC Bournemouth,English Premier League (1)
436,J. Anthony,Jaidon Anthony,AFC Bournemouth,English Premier League (1)
437,J. Lowe,Jamal Lowe,AFC Bournemouth,English Premier League (1)
438,C. Mepham,Chris Mepham,AFC Bournemouth,English Premier League (1)
...,...,...,...,...
337,Daniel Podence,Daniel Castelo Podence,Wolverhampton Wanderers,English Premier League (1)
338,M. Kilman,Max Kilman,Wolverhampton Wanderers,English Premier League (1)
339,Hwang Hee Chan,Hee Chan Hwang,Wolverhampton Wanderers,English Premier League (1)
333,Adama Traoré,Adama Traoré Diarra,Wolverhampton Wanderers,English Premier League (1)


In [349]:
# Get the data from API

def scrape_whoscored(id, homeTeam, awayTeam):
    url = f"https://api.sofascore.com/api/v1/event/{id}/lineups"
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
    r = requests.get(url, headers=headers)
    json_data = r.json()

    home_players_json = json_data["home"]["players"]
    away_players_json = json_data["away"]["players"]

    ls = []

    for player in home_players_json:
        ls.append((player["player"]["name"], homeTeam))

    for player in away_players_json:
        ls.append((player["player"]["name"], awayTeam))
        
    return ls


In [805]:
def flatten(xss):
    return [x for xs in xss for x in xs]

def Convert(tup, di):
    for b, a in tup:
        di.setdefault(a, []).append(b)
    return di

def goThroughGameDay(start_date, days):
    
    ids = {}
    for day in range(days):
        when = datetime.datetime.strptime(start_date, '%Y-%m-%d').date() - timedelta(days=day)
        #print(when)
        url = f"https://api.sofascore.com/api/v1/sport/football/scheduled-events/{when}"
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
        r = requests.get(url, headers=headers)
        json_data = r.json()

        

        for x in json_data['events']:
            if(x['tournament']['name'] == 'Premier League' and x['tournament']['category']['name'] == 'England') \
                and x['id'] not in ids:
                ids[x['id']] = (x['homeTeam']['name'], x['awayTeam']['name'])

        del json_data

    allPlayers = []
    for i in ids.keys():
        allPlayers.append(scrape_whoscored( i, ids[i][0], ids[i][1] ))
        #add wait sleep(5)
    
    tup = list(set(flatten(allPlayers)))
    dc = {}

    di = Convert(tup, dc)

    return (pd.DataFrame([(key, var) for (key, L) in di.items() for var in L], 
                 columns=['team_name', 'player']))


In [ ]:
goThroughGameDay('2022-05-15', 6)

,team_name,player
0,Watford,Oghenekaro Etebo
1,Watford,Samir
2,Watford,Ben Foster
3,Watford,Adam Masina
4,Watford,Kwadwo Baah
...,...,...
345,Brentford,Christian Eriksen
346,Brentford,Ivan Toney
347,Brentford,Joshua Dasilva
348,Brentford,Finley Stevens


In [437]:
all_players_df = goThroughGameDay('2022-05-22', 6)

In [362]:
#scrape_whoscored("9576298", "Tottenham Hotspur", "Burnley")

[('Hugo Lloris', 'Tottenham Hotspur'),
 ('Davinson Sánchez', 'Tottenham Hotspur'),
 ('Eric Dier', 'Tottenham Hotspur'),
 ('Ben Davies', 'Tottenham Hotspur'),
 ('Emerson Royal', 'Tottenham Hotspur'),
 ('Rodrigo Bentancur', 'Tottenham Hotspur'),
 ('Pierre-Emile Højbjerg', 'Tottenham Hotspur'),
 ('Ryan Sessegnon', 'Tottenham Hotspur'),
 ('Lucas Moura', 'Tottenham Hotspur'),
 ('Heung-min Son', 'Tottenham Hotspur'),
 ('Harry Kane', 'Tottenham Hotspur'),
 ('Pierluigi Gollini', 'Tottenham Hotspur'),
 ('Brandon Austin', 'Tottenham Hotspur'),
 ('Joe Rodon', 'Tottenham Hotspur'),
 ('Matthew Craig', 'Tottenham Hotspur'),
 ('Harry Winks', 'Tottenham Hotspur'),
 ('Harvey White', 'Tottenham Hotspur'),
 ('Dejan Kulusevski', 'Tottenham Hotspur'),
 ('Steven Bergwijn', 'Tottenham Hotspur'),
 ('Dane Scarlett', 'Tottenham Hotspur'),
 ('Nick Pope', 'Burnley'),
 ('Connor Roberts', 'Burnley'),
 ('Kevin Long', 'Burnley'),
 ('Nathan Collins', 'Burnley'),
 ('Matthew Lowton', 'Burnley'),
 ('Charlie Taylor', 'Bur

In [ ]:
def in_name2(player, team_name, fpl_df):
    ls = []
    for x in range(len(fpl_df['full_name'][fpl_df['team_name'] == team_name])):
        if (fuzz.token_set_ratio(fpl_df['full_name'][fpl_df['team_name'] == team_name].iloc[x], player)) >= 88:
            ls.append(fpl_df['web_name'][fpl_df['team_name'] == team_name].iloc[x])
    if (len(ls)>0):
        return ls[0]
    else:
        return ''


In [768]:
all_players_df['known_as'] = all_players_df.apply(lambda x: in_name2(x['player'], x['team_name']), axis=1)

In [785]:
all_players_df.sample(11)

,team_name,player,known_as
43,Chelsea,Ben Chilwell,Chilwell
102,Brighton & Hove Albion,Enock Mwepu,Mwepu
345,Tottenham Hotspur,Harry Kane,Kane
305,Wolverhampton,Pedro Neto,Neto
390,Leeds United,Charlie Cresswell,
128,Arsenal,Takehiro Tomiyasu,Tomiyasu
335,Watford,Cucho Hernández,
217,Aston Villa,Douglas Luiz,Douglas Luiz
343,Tottenham Hotspur,Harry Winks,Winks
349,Tottenham Hotspur,Pierluigi Gollini,


In [786]:
all_players_df.to_csv('players2.csv')

In [792]:
all_players_df[all_players_df['team_name'] == 'Tottenham Hotspur']

,team_name,player,known_as
340,Tottenham Hotspur,Ben Davies,Davies
341,Tottenham Hotspur,Eric Dier,Dier
342,Tottenham Hotspur,Rodrigo Bentancur,Bentancur
343,Tottenham Hotspur,Harry Winks,Winks
344,Tottenham Hotspur,Alfie Devine,Devine
345,Tottenham Hotspur,Harry Kane,Kane
346,Tottenham Hotspur,Dejan Kulusevski,Kulusevski
347,Tottenham Hotspur,Harvey White,White
348,Tottenham Hotspur,Hugo Lloris,Lloris
349,Tottenham Hotspur,Pierluigi Gollini,
